# Load data

In [ ]:
import pandas as pd

# Load the spreadsheet into a Pandas data frame
df_pt = pd.read_excel("data/Bus_patronage.xlsx")

# View some of the data to check what it looks like - head displays some rows from the top of the table
df_pt.head()

In [ ]:
# show number of not null rows for each field
df_pt.count()

# Filtering

In [ ]:
# Select into a new data frame
df_pt_filtered = df_pt[(df_pt["Route"] == 33) & (df_pt["Direction"] == 1)]

# tail displays some rows from the end of the table
df_pt_filtered.tail()

# Simple calculations

In [ ]:
# Simple transformations
df_pt["Patrons*dist (m)"] = df_pt["Passengers Onboard"] * df_pt["Distance from Prev Stop"]

df_pt.head()

# Advanced calculations/logic

In [ ]:
def get_peak_hour_approx(time):
    """Return values for 8AM-9:15AM as morning peak, 5PM-6:15PM as evening peak"""
    # time was automatically parsed into a datetime.time object
    hour = time.hour
    minute = time.minute
    if hour == 8 or (hour == 9 and minute <= 15):
        return "AM"
    if hour == 17 or (hour == 18 and minute <= 15):
        return "PM"
    # anything not matching the above will default to None


# Apply the function to the field
df_pt["Peak period"] = df_pt["Stop Minute (Sched)"].apply(get_peak_hour_approx)

# filter out the "NaN" results (i.e. remove rows not in a peak period)
df_pt_peak = df_pt[df_pt["Peak period"].notna()]

df_pt_peak.tail()

# Aggregation

In [ ]:
# Group and aggregate (as_index=False keeps the output "flat", with a single header)
df_pt_grouped = df_pt_peak.groupby(
    ["Route", "Direction", "Peak period"], as_index=False
).agg({"Distance from Prev Stop": "sum", "Patrons*dist (m)": "sum"})

df_pt_grouped["Weighted patronage"] = (
    df_pt_grouped["Patrons*dist (m)"] / df_pt_grouped["Distance from Prev Stop"]
)

# Show more rows with head
df_pt_grouped.head(n=20)

# Plotting

In [ ]:
# minimise data so we don't overload the plot
df_pt_grouped_am = df_pt_grouped[(df_pt_grouped["Peak period"] == "AM")
    & (df_pt_grouped["Direction"] == 1) & (df_pt_grouped["Route"] < 20)
]

# set Route column as the "index"
df_pt_grouped_am.set_index("Route", inplace=True)

# bar plot the weighted patronage for each route
df_pt_grouped_am[["Weighted patronage"]].plot(
    kind="bar", ylabel="Weighted patronage", xlabel="Route"
)

# Export

In [ ]:
df_pt_grouped.to_excel("temp.xlsx")